In [1]:
import os
# os.environ['KERAS_BACKEND'] = 'tensorflow'
import keras
import numpy as np
import csv, json
from zipfile import ZipFile
from os.path import expanduser, exists
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.data_utils import get_file
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation

/share/apps/anaconda3-5.0.1/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:

Train_QUESTION_PAIRS_FILE = '../../SO_data/Last_Medium_noClue/medium_link_prediction_noClue_shuffled_train.csv'
Test_QUESTION_PAIRS_FILE =  '../../SO_data/Last_Medium_noClue/medium_link_prediction_noClue_shuffled_test.csv'
GLOVE_FILE = 'glove.840B.300d.txt'
MAX_NB_WORDS = 146090
MAX_SEQUENCE_LENGTH_body = 70
EMBEDDING_DIM = 300

In [4]:
import re
import string
import Levenshtein


def convertCamelCase(name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    finals = re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()
    #     print(finals)
    return finals


def text_to_wordlist(text, remove_stopwords=True, stem_words=False, remove_punc=True):
    # URL remove
    #     print(type(text))
    text = str(text)
    text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', 'URL', text)
    # split words:
    text = text.split()
    text2 = []
    for t in text:
        splited = re.split("[.,' \-!?:#^%*[$\]+|_`\)=<;{\"&>@/~(\\}\\\]+", t)
        text2.extend(splited)
    not_digits=[]    
    for i in text2:
        if i.isdigit():
            not_digits.append("cc")
        else:
            not_digits.append(i)
    text4 = []
    for c in not_digits:
        if len(c) > 1:
            if c[1].islower():
                for k in c[1:]:
                    if not k.islower():
                        text4.append(convertCamelCase(c))
                        break;
                else:
                    text4.append(c)
            else:
                text4.append(c)
        else:
            text4.append(c)

    text5 = []
    for t in text4:
        splited = re.split("_", t)
        text5.extend(splited)
    text6=[]
    for x in text5:
        if len(x)>1:
            text6.append(x)

    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text6 if not w in stops]
    # print("after stop ", text)
    # print("after stop word", text)
    text = " ".join(text)

    text = re.sub("  ", " ", text)
    text = re.sub("   ", " ", text)
    text = re.sub("    ", " ", text)
    text = re.sub("[0-9]+", "CC", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"couldn't", "could not ", text)
    text = re.sub(r"doesn’t", "does not ", text)
    text = re.sub(r"don't", "do not ", text)
    text = re.sub(r"won't", "will not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'", "", text)
    text = re.sub(r"\"", "", text)
    text = re.sub(r"“", "", text)
    text = re.sub(r"”", "", text)

    if remove_punc:
        exclude = set(string.punctuation)
        text = ''.join(ch for ch in text if ch not in exclude)

    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)

    text = text.lower()

    return (text)

In [5]:
test = text_to_wordlist("a5HRsatX is \"qood\" or 'WOOd or EooOd or rooOood AaokkE ยง15 !?,,'[]{} to a the - _", remove_stopwords=True, remove_punc=True)
print(test)

acchrsatx qood wood eoo od roo oood aaokk ยงcc


In [6]:
train_q1_Id =[]
train_q1_Title =[]
train_q1_Body =[]
train_q1_AcceptedAnswerId =[]
train_q1_AcceptedAnswerBody=[]
train_q1_AnswersIdList=[]
train_q1_AnswersBody=[]
train_q1_Tags=[]
train_q2_Id=[]
train_q2_Title=[]
train_q2_Body=[]
train_q2_AcceptedAnswerId=[]
train_q2_AcceptedAnswerBody=[]
train_q2_AnswersIdList=[]
train_q2_AnswersBody=[]
train_q2_Tags=[]
train_class=[]

import pandas as pd
df = pd.read_csv( Train_QUESTION_PAIRS_FILE, sep=",", lineterminator='\n' )
df.columns=['id', 'q1_Id', 'q1_Title', 'q1_Body', 'q1_BodyCode','q1_AcceptedAnswerId', 'q1_AcceptedAnswerBody', 'q1_AcceptedAnswerCode', 'q1_AnswersIdList', 'q1_AnswersBody', 'q1_AnswersCode', 'q1_Tags', 'q2_Id', 'q2_Title', 'q2_Body', 'q2_BodyCode', 'q2_AcceptedAnswerId', 'q2_AcceptedAnswerBody', 'q2_AcceptedAnswerCode', 'q2_AnswersIdList', 'q2_AnswersBody', 'q2_AnswersCode', 'q2_Tags', 'class' ]
count =0
for index, row in df.iterrows():
    q1_Body = text_to_wordlist(str(row['q1_Title'])+ " " +str(row['q1_Body']), remove_stopwords=True, stem_words=False)
    train_q1_Body.append(q1_Body)

    q2_Body = text_to_wordlist(str(row['q2_Title'])+ " " +str(row['q2_Body']), remove_stopwords=True, stem_words=False)
    train_q2_Body.append(q2_Body)
    train_class.append(row['class'])
    
    count+=1
print(count)

32000


In [7]:
import _pickle as pk
preprocessed_data = "preprocessed_data_noClue_T+B_medium/"
    
with open(preprocessed_data+"train_q1_Body.pkl", 'wb') as fb:
    pk.dump(train_q1_Body, fb)

with open(preprocessed_data+"train_q2_Body.pkl", 'wb') as fb:
    pk.dump(train_q2_Body, fb)

with open(preprocessed_data+"train_class.pkl", 'wb') as fb:
    pk.dump(train_class, fb)

print("train_class ", train_class[:2])


train_class  ['duplicate\n', 'duplicate\n']


In [8]:
test_q1_Id =[]
test_q1_Title =[]
test_q1_Body =[]
test_q1_AcceptedAnswerId =[]
test_q1_AcceptedAnswerBody=[]
test_q1_AnswersIdList=[]
test_q1_AnswersBody=[]
test_q1_Tags=[]
test_q2_Id=[]
test_q2_Title=[]
test_q2_Body=[]
test_q2_AcceptedAnswerId=[]
test_q2_AcceptedAnswerBody=[]
test_q2_AnswersIdList=[]
test_q2_AnswersBody=[]
test_q2_Tags=[]
test_class=[]

import pandas as pd
df = pd.read_csv(Test_QUESTION_PAIRS_FILE, sep=",", lineterminator='\n' )
df.columns=['id', 'q1_Id', 'q1_Title', 'q1_Body', 'q1_BodyCode','q1_AcceptedAnswerId', 'q1_AcceptedAnswerBody', 'q1_AcceptedAnswerCode', 'q1_AnswersIdList', 'q1_AnswersBody', 'q1_AnswersCode', 'q1_Tags', 'q2_Id', 'q2_Title', 'q2_Body', 'q2_BodyCode', 'q2_AcceptedAnswerId', 'q2_AcceptedAnswerBody', 'q2_AcceptedAnswerCode', 'q2_AnswersIdList', 'q2_AnswersBody', 'q2_AnswersCode', 'q2_Tags', 'class' ]
count =0
for index, row in df.iterrows():

    q1_Body = text_to_wordlist(str(row['q1_Title'])+ " " +str(row['q1_Body']), remove_stopwords=True, stem_words=False)
    test_q1_Body.append(q1_Body)

    q2_Body = text_to_wordlist(str(row['q2_Title'])+ " " +str(row['q2_Body']), remove_stopwords=True, stem_words=False)
    test_q2_Body.append(q2_Body)

    test_class.append(row['class'])
    
    count+=1
print(count)

8000


In [9]:

    
with open(preprocessed_data+"test_q1_Body.pkl", 'wb') as fb:
    pk.dump(test_q1_Body, fb)


with open(preprocessed_data+"test_q2_Body.pkl", 'wb') as fb:
    pk.dump(test_q2_Body, fb)

with open(preprocessed_data+"test_class.pkl", 'wb') as fb:
    pk.dump(test_class, fb)

print("test_class ", test_class[:2])

test_class  ['duplicate\n', 'isolated\n']


In [13]:
import _pickle as pk
preprocessed_data = "preprocessed_data_noClue_T+B_medium/"
    
train_q1_Body= pk.load(open(preprocessed_data+"train_q1_Body.pkl", 'rb'))

train_q2_Body= pk.load(open(preprocessed_data+"train_q2_Body.pkl", 'rb'))   

train_labels= pk.load(open(preprocessed_data+"train_class.pkl", 'rb'))

In [14]:
train_labels= pk.load(open(preprocessed_data+"train_class.pkl", 'rb'))

In [15]:

    
with open(preprocessed_data+"test_q1_Body.pkl", 'rb') as fb:
    test_q1_Body= pk.load(fb)

with open(preprocessed_data+"test_q2_Body.pkl", 'rb') as fb:
    test_q2_Body= pk.load(fb)
    
with open(preprocessed_data+"test_class.pkl", 'rb') as fb:
    test_labels= pk.load(fb)

In [16]:
with open(preprocessed_data+"test_class.pkl", 'rb') as fb:
    test_labels= pk.load(fb)

## Build tokenizer

In [26]:
questions =  test_q1_Body +  \
test_q2_Body +  \
 train_q1_Body + \
train_q2_Body  
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(questions)
train_q1_Body_word_sequences = tokenizer.texts_to_sequences(train_q1_Body)
train_q2_Body_word_sequences = tokenizer.texts_to_sequences(train_q2_Body)
test_q1_Body_word_sequences = tokenizer.texts_to_sequences(test_q1_Body)
test_q2_Body_word_sequences = tokenizer.texts_to_sequences(test_q2_Body)

word_index = tokenizer.word_index
print("Words in index: %d" % len(word_index))
print(test_q2_Body_word_sequences[0])

Words in index: 51109
[3315, 10643, 826, 145, 347, 3315, 441, 2242, 564, 159, 149, 104, 6, 46, 243, 274, 183, 3315, 564, 8574, 3869, 5, 187, 443, 515, 136, 822, 2231, 389, 170, 186, 3315, 87, 184, 2286, 3869, 995, 848, 54, 34, 78, 182, 74, 658, 1053, 509, 167, 37, 798, 4249, 1818, 4105, 274, 29, 75, 59, 500, 17, 1773]


In [27]:
EMBEDDING_DIM = 300
print("Processing", GLOVE_FILE)

embeddings_index = {}
with open('../../../embedding/'+GLOVE_FILE, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings: %d' % len(embeddings_index))

Processing glove.840B.300d.txt
Word embeddings: 2196016


## GLOVE

In [28]:
nb_words = min(MAX_NB_WORDS, len(word_index))
word_embedding_matrix = np.zeros((nb_words + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    unknown_vector = embeddings_index.get("java")
    if embedding_vector is not None:
        word_embedding_matrix[i] = embedding_vector
    else:
#         word_embedding_matrix[i] =unknown_vector
        print(word)

print('Null word embeddings: %d' % np.sum(np.sum(word_embedding_matrix, axis=1) == 0))

logccj
strutscc
ccxcc
jdkcc
testcc
slfccj
jsoup
ioexception
ccnd
ccrd
fxml
ccst
jlabel
sqlexception
ccmb
mpcc
pdfbox
ccth
jsonobject
jcombobox
ljava
basecc
tomcatcc
ccucc
ccgb
editcc
wildfly
resteasy
ccbit
logccjcc
javase
httpcc
listcc
jooq
jdbccc
kotlin
ccms
jtextarea
jrecc
externalizable
jscrollpane
jccee
jdialog
joptionpane
urlclassloader
junitcc
openqa
assdet
thymeleaf
cctcc
mdcc
ipvcc
jpql
classcc
jpanels
pojos
axiscc
neoccj
sqlserverconnection
invokecc
ftpclient
fasterxml
jsonarray
usercc
sqlserverexception
methodcc
jtextpane
pkcscc
oauthcc
mybatis
ccfcc
urlconnection
cckb
cccpcc
projectcc
htmlunit
arraylists
jfilechooser
valuecc
logcc
asynctask
i’m
fieldcc
graphicscc
wccc
shacc
utfcc
jtabbedpane
djava
jasperreports
hibernatecc
openshift
jvms
ojdbccc
jbuttons
nashorn
stringcc
sqlerror
eccapplication
antlrcc
jprogressbar
varcc
rccc
sqlserverdriver
ccz
mcce
jslider
â€
itextpdf
systemcc
hamcrest
gralloc
sscce
sqljdbccc
namecc
threadcc
unitcc
jpmml
assetaddcc
updatecc
ognl
jaxrs
quer

ttransportexception
sconsonantarray
dukpt
jtextfiled
jonathanbenoudiz
superpackages
hasuser
cfgfile
sspace
canvasmaker
xsddto
xnio
mingwcc
udccd
bigintegers
staxb
hsluo
geoline
mywebservice
casquejo
loginadmin
cccaccccc
consolab
ttfs
sopln
modelmbean
cxmessenger
that’s
ctoolkit
chimmi
ackcc
nackcc
clistcc
fge
gccct
coördinates
sqlserverpreparedstatement
greenrobot
wrappercc
myprojects
fermate
asdflas
uclient
ccxccaa
exportcert
stickmanccleft
sqlexceptiontypedelegate
cadenaser
etranslate
avantaj
timestamper
conditioncc
webresources
supercsv
schemacc
jcajce
angularcc
soapenvelope
esccresourcefactory
metawidget
τοῦ
да
sidecc
bgcc
cclcom
datechooser
testusr
ccgy
nccc
progressdialog
statsd
babbly
mifos
stateccof
stsclient
scolorq
oncreate
githubusercontent
gpsstatus
someonoe
calculatorcc
ccfusers
ccfhome
ccfdev
ccfapp
ccfappprj
ccfadapters
ccfadapter
wdllwdwwwlldd
wllddwllddwlldd
wlldd
sequenceinputstream
sslserversocket
managedbean
kadane
dasblinkenlight
jregistrykey
depedencies
usbcc
sapu

liberaries
bisual
outputtext
motable
pdist
dtar
appodeal
tabpane
getinfowindow
stringpainted
happears
февраль
sshpass
androidapplication
testgame
patternstring
adfa
foreignkey
iauthorizationstrategy
lã³pez
consutrct
mavenish
ljpcap
xmlwriter
findbyname
wmftranscoder
bdanielcc
spoonloader
ejbcontext
wsjaas
csivcc
vraptor
addcc
systables
zentus
ccbdeccdecc
rdbm
ccecceccbaccccaccacccdccccc
atrributes
stdfccj
stdfparser
imageccup
khoy
renderthis
projektdetail
costrunctor
testmachine
oauthccauthorizationconfiguration
fileservlet
nummbers
javabuild
mwnumericarray
ccjsrs
abcdecc
clieng
listfield
ecccaaccecc
cceccfaccacc
doublecc
nativebases
logonerror
saveddata
okserver
cccontent
teachercc
hibenate
massistupdatebutton
tablecolumns
cellof
mulltiply
xmllexer
hfileoutputformatcc
scep
ejbca
sentrick
dtnode
malfuntions
ccnqb
szfmgz
windowcccom
they’re
myfregefunction
jzebra
someparam
commonname
proddetail
ipccw
tacc
dbcursor
jongo
smartgwt
hcwcc
�����
locationcc
droidio
bufferreader
intermitency
m

nexusâ„¢
sponsershipe
sponership
strinr
explot
decisioncc
wlserver
jpaknowledgeservice
ccccckp
jtcczv
jkxp
wampus
swatchservice
shellsort
tcompactprotocol
libzccjava
memorystream
jcip
sdroidmsg
laylout
clazzname
lsmyhost
isuserloggedin
pcc�ep
��qddgp
�en
pjep
ã•zbek
flowbits
cceccadbfcc
housescc
ffmain
gamescreen
glashfish
eqqbccyccmexiccc
larguero
cccsrcsrdep
gke
fluentd
eawt
perfarmance
mememory
nric
scatterred
runningâ€¦
multibindings
aritrarty
datadase
storecc
testimage
whatprovides
sessionscoped
requestscoped
sessioncontroller
sessionobject
myns
–jar
containned
ckrecording
javaccdsloader
werror
sticher
cchours
javaâ€™s
cccn
redrawed
schemalocation
pdmxqeagdxsccsowpercchqjqfcce
beforemethod
testedannotation
iworkbenchthemeconstants
expoet
hostinterface
egating
medexter
imporovement
platformrequest
rulecc
illegalstatexception
htmldivelement
ffcceac
lowestguess
mthmulders
someinfo
rsacrypto
sebsauvage
ccfccbccdbccdcc
sducc
vxpuxaccaxoccfpgksfccnccrccdzdcclwqkvroye
beanname
smanager
u

condigured
mymojo
jesrey
usingheader
vvadin
wsproviderconfig
wssccjresourcebundle
libreofice
networkon
mjrusso
dmyprop
csvread
keytstore
avariable
tcts
fpms
jsoneditoronline
stdoutput
jwts
unamanic
accebcccccbccfbccccccbccbccd
operandcc
pdfparserconfig
bcpkix
linkedqueue
codenameone
folliwing
svstore
aaxo
xoaa
mpjpeg
bitsream
bitsreams
referens
splashport
jvmparams
trccing
documccntation
steelbrothers
lifeifei
knnj
binarysearch
itextgoogle
droppages
document：
text、annotation
case：
textitem
textpaint
glthread
anotated
duccproject
ducc
pavlovský
pavlovsk
íáýěéáěř
servilet
modolu
mmsshh
scrued
newtest
rccprecc
httpparams
दर्शन
à¤¦à¤°à¥à¤¶à¤¨
getfrom
addresslist
otherâ€™s
gotw
millcc
graphx
edgeclass
webjars
diretcory
testconfig
zhengyecc
llya
rtypeid
sawadaptercontext
ojones
coldmail
ocrabc
ccmegabytes
libsome
realod
tfield
formulacc
jsonobect
makred
agregating
pradyut
gallerycc
atyrau
f¿r
somesong
addpart
socketexception
uninheritable
artifactoy
classattribute
secondstocopy
millisecondst

ebfu
countains
junitexception
javapos
cccccfcccbccbeccbcc
querypart
ctci
dispacther
lohit
kccvmx
sublselect
totalout
suselected
fcccccdcceccaacccacccccaccbccfcc
ccmill
suvojit
dootherthing
averagewithmethods
‘advanced’
example������
درود����������������
intheend
umxccayvi
ccwnzcch
specefies
charactedr
senab
distravel
synchroization
coutdowntimer
webreports
ccbccccc
ssls
unsafetiness
errorpage
sitnikov
doughtnut
anyb
mohamadjaber
includeapp
reosurce
cpccr
pcctatgrccukm
maintarget
altstdout
exaact
scraoe
antlrs
subbarao
failurescc
gleventlistener
alement
gettins
msysobjects
recipedetails
statful
bisr
foreignkeys
marchccst
startqcc
oauthtokencredential
microproc
foobarcc
pushig
additoin
soapmessages
prefic
classes’
tessract
jmenubars
ontotoly
bonigarcia
webdrivermanager
auscc
gcccc
ccclibpulse
kjhughes
buttions
jpanelarray
typestate
mancook
tutorcc
mdocument
safenet
networkonmainthreadexception
ccatrue
dsgdsb
icecc
manaegement
cchave
ccbeen
cclogged
ccout
elasticserch
elasticsreach
ccxidw

headerfile
projectdetails
concurently
ioutil
wlsfullclient
ifacade
funcktion
rolegets
nlwp
ccxccfccccc
ccxccccca
ccxccfccbbccecc
ccxccdcccccfcc
ccxccfcccccccc
ccxccfcccaccacc
jexec
taskadapter
sorokin
sortor
x�͝o�
�q���
���e�d�a��cc
h��ǃ
ns��
��������
���x
��ri��j����y�r��f�
postgers
preservring
resucively
angularj
unsimplfied
loã¯s
jrjdbcqueryexecuter
charlesvigneault
exxx
surefireplugin
logmanager
oprerator
hlccvcc
callbacksuccess
ldcc
comess
mqgmo
pdftextstripperbyarea
pdgraphicsstate
libart
gdvm
libdvm
chatclient
httpunit
internationnalizing
anvard
ariticles
decouplling
cczccbs
colspec
ccmilestone
ccpriority
ccstatus
ccsummary
serers
iunknown
ipersiststream
iolewindow
icontextmenu
iclassfactory
xzy
rezipping
iastate
latn
cyrl
valides
configuree
strto
eccxmi
kosnyru
mthumbsids
mthumbsid
behaior
textvakcc
cennector
ccxccfcceccfcc
ccxdccd
ccxccfccfccc
keyur
cvtcolor
mgray
bgraccgray
sepratly
internationalizaiton
vincentramdhanie
contstants
mailtemplates
guiodrehandler
guihentordre
blo

## padding

In [29]:

print('Shape of train question2 data tensor:', len(train_q1_Body_word_sequences))
print('Shape of train question2 data tensor:', len(train_q2_Body_word_sequences))

print('Shape of train question2 data tensor:', len(test_q1_Body_word_sequences))

print('Shape of train question2 data tensor:', len(test_q2_Body_word_sequences))

q1_train_body = pad_sequences(train_q1_Body_word_sequences, maxlen=MAX_SEQUENCE_LENGTH_body)

q2_train_body = pad_sequences(train_q2_Body_word_sequences, maxlen=MAX_SEQUENCE_LENGTH_body)

q1_test_body = pad_sequences(test_q1_Body_word_sequences, maxlen=MAX_SEQUENCE_LENGTH_body)

q2_test_body = pad_sequences(test_q2_Body_word_sequences, maxlen=MAX_SEQUENCE_LENGTH_body)


print("After padding . .. . ")
print('Shape of train question2 data tensor:', q1_train_body.shape)

print('Shape of train question2 data tensor:', q2_train_body.shape)

print('Shape of train question2 data tensor:', q1_test_body.shape)

print('Shape of train question2 data tensor:', q2_test_body.shape)




Shape of train question2 data tensor: 32000
Shape of train question2 data tensor: 32000
Shape of train question2 data tensor: 8000
Shape of train question2 data tensor: 8000
After padding . .. . 
Shape of train question2 data tensor: (32000, 70)
Shape of train question2 data tensor: (32000, 70)
Shape of train question2 data tensor: (8000, 70)
Shape of train question2 data tensor: (8000, 70)


## file names:

In [30]:
weight ="glove_weight1060180_nb=14609_T+B_medium/"

q1_train_body_FILE =weight+"q1_train_body_FILE.npy"

q2_train_body_FILE =weight+"q2_train_body_FILE.npy"

q1_test_body_FILE =weight+"q1_test_body_FILE.npy"

q2_test_body_FILE =weight+"q2_test_body_FILE.npy"


train_labels_FILE =weight+"train_labels_FILE.npy"
test_labels_FILE =weight+"test_labels_FILE.npy"
# dev_labels_FILE =weight+"dev_labels_FILE.npy"

WORD_EMBEDDING_MATRIX_FILE = weight+"WORD_EMBEDDING_MATRIX_FILE.npy"
NB_WORDS_DATA_FILE = weight+'nb_words.json'

In [31]:

print(test_labels[:2])
print("test class ", len(test_labels))
print("train class ", len(train_labels))
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit(train_labels[:40])
train_labels = lb.transform(train_labels)


lbT = preprocessing.LabelBinarizer()
lbT.fit(test_labels[:40])
test_labels = lb.transform(test_labels)


print('Shape of label tensor:', train_labels.shape)
print('Shape of label tensor:', test_labels.shape)

print(test_labels[:5])

[[0 1 0 0]
 [0 0 0 1]]
test class  8000
train class  32000
Shape of label tensor: (32000, 4)
Shape of label tensor: (8000, 4)
[[0 1 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 1 0]]


## Saving

In [32]:

np.save(open(q1_train_body_FILE, 'wb'), q1_train_body)

np.save(open(q2_train_body_FILE, 'wb'), q2_train_body)

np.save(open(q1_test_body_FILE, 'wb'), q1_test_body)

np.save(open(q2_test_body_FILE, 'wb'), q2_test_body)

np.save(open(train_labels_FILE, 'wb'), train_labels)
np.save(open(test_labels_FILE, 'wb'), test_labels)


np.save(open(WORD_EMBEDDING_MATRIX_FILE, 'wb'), word_embedding_matrix)

with open(NB_WORDS_DATA_FILE, 'w') as f:
    json.dump({'nb_words': nb_words}, f)
